#### 1. Importing all the libraries

In [1]:
import pandas as pd
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import seaborn as sns
import sys
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn import tree
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

#### 2. Define the Functions

In [2]:
# Function that create the episode data - sample randomaly
def get_data(episode_size,policy,mode):
    global dataset
    if mode=='train':
        if policy==0:
             dataset=data.sample(n=episode_size)
        else:
            dataset=data
    else:
        dataset = pd.read_csv(location + '/' + file +'_test_int.csv', index_col=0)
    return dataset

In [3]:
# Function that separate the episode data into features and label
def data_separate (dataset):
    global X
    global y    
    X = dataset.iloc[:,0:dataset.shape[1]-1]  # all rows, all the features and no labels
    y = dataset.iloc[:, -1]  # all rows, label only
    return X,y

In [4]:
# Function that split the episode data into train and test
def data_split(X,y):
    global X_train_main
    global X_test_main   
    global y_train
    global y_test  
    from sklearn.model_selection import train_test_split
    X_train_main, X_test_main, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=4)
    return X_train_main, X_test_main, y_train, y_test

In [5]:
# Function that chooses exploration or explotation method
def exploration_explotation(epsilon):
    global exploration 
    if np.random.rand() < epsilon:  
        exploration=1
    else:
        exploration=0    
    return exploration

In [6]:
# Function that returns all available actions in the state given as an argument: 
def available_actions(number_of_columns,columns,initial_state,current_state,trashold, exploration):
    global exclude
    global all_columns
#    exclude=[]
    all_columns=np.arange(number_of_columns+1)
    # remove columns that have been already selected
    exclude=columns.copy()
    # remove the initial_state and the current_state
    exclude.extend([initial_state, current_state])
    available_act = list(set(all_columns)-set(exclude))
    # remove actions that have negetiv Q value
    if exploration==0:
        index = np.where(Q[current_state,available_act] > trashold)[1]
        available_act= [available_act[i] for i in index.tolist()]
    return available_act

In [7]:
# Function that chooses which action to be performed according to exploration - explotation method
def sample_next_action(current_state, Q, available_act, exploration):
    global available_act_q_value
    available_act_q_value=np.array(Q[current_state,available_act]).reshape(-1,).tolist()
    if exploration==1: 
        #random selection
        next_action = int(np.random.choice(available_act,1))
    else: 
        #greedy selection according to max value 
        maxQ=max(available_act_q_value)
        count = available_act_q_value.count(maxQ)
        if count > 1:
            max_columns =[i for i in range(len(available_act_q_value)) if available_act_q_value[i] == maxQ]
            i = np.random.choice(max_columns)
        else:
            i = available_act_q_value.index(maxQ)
        next_action=available_act[i]        
    return next_action

In [8]:
# function that update a list with all selected columns in the episode
def update_columns(action, columns):
    update_columns=columns
    update_columns.append(action)
    return update_columns

In [9]:
# function that update the X_train and X_test according to the current episode columns list 
def update_X_train_X_test(columns,X_train_main, X_test_main):
    X_train=X_train_main.iloc[:,columns]
    X_test=X_test_main.iloc[:,columns]
    X_train=pd.DataFrame(X_train)
    X_test=pd.DataFrame(X_test)
    return X_train, X_test

In [10]:
# Function that run the learner and get the error to the current episode columns list
def Learner(X_train, X_test,y_train, y_test):
    global learner
    global y_pred
    if learner_model == 'DT':
        learner = tree.DecisionTreeClassifier()
        learner = learner.fit(X_train, y_train)
        y_pred = learner.predict(X_test)
    elif learner_model == 'KNN':
        learner = KNeighborsClassifier(metric='hamming',n_neighbors=5)
        learner = learner.fit(X_train, y_train)
        y_pred = learner.predict(X_test)        
    elif learner_model == 'SVM':
        learner = SVC()
        learner = learner.fit(X_train, y_train)
        y_pred = learner.predict(X_test)        
    elif learner_model == 'NB':
        learner = MultinomialNB()
        learner = learner.fit(X_train, y_train)
        y_pred = learner.predict(X_test)
    elif learner_model == 'AB':
        learner = AdaBoostClassifier()
        learner = learner.fit(X_train, y_train)
        y_pred = learner.predict(X_test)  
    elif learner_model == 'GB':
        learner = GradientBoostingClassifier()
        learner = learner.fit(X_train, y_train)
        y_pred = learner.predict(X_test)  
    accuracy=metrics.accuracy_score(y_test, y_pred)
    error=1-accuracy
    return error

In [11]:
# Function that updates the Q matrix according to the path selected and the Q 
def q_update(current_state ,action,learning_rate, reward):
    # next_state = current action
    max_index = np.where(Q[action,] == np.max(Q[action,]))[1]    
    if max_index.shape[0] > 1:
        #np.random.seed(seed)
        max_index = int(np.random.choice(max_index, size = 1))
    else:
        max_index = int(max_index)
    max_value = Q[action, max_index]
    #we start with 1 for all Q values and update with reward only at the 1st time
    if Q[current_state, action]==1:
        Q[current_state, action] = learning_rate*reward
    else:
         Q[current_state, action] = Q[current_state, action]+ learning_rate*(reward + (discount_factor * max_value)-Q[current_state, action])
    #Q[current_state, action] = Q[current_state, action]+ learning_rate*(reward + (discount_factor * max_value)-Q[current_state, action])

### Experiment mangment

#### 3. Define the parameters 

In [12]:
#Experiment: 
experiment='new_No_show_DT_2'
number_of_experiment=5
# Dataset parameters #
location = 'C:/Users/mmoran1/Documents/TAU/Thesis/Datasets'
outputlocation='C:/Users/mmoran1/Documents/TAU/Thesis'
file='No_show' #adult #diabetic_data #Census_Income_KDD
#np.random.seed(3)

# Q learning parameter # 
learning_rate=0.005
discount_factor=0.01
epsilon = 0.1

# Learner and episode parameters #
learner_model = 'DT' #DT #KNN #SVM
episode_size=100
internal_trashold=0
external_trashold=0
filename= file +'_int.csv'

#Experiments folder management: 
if not os.path.exists('/Experiments'):
    os.makedirs('/Experiments') 
if not os.path.exists('Experiments/'+ str(experiment)):
    os.makedirs('Experiments/'+ str(experiment))
else:
    shutil.rmtree('Experiments/'+ str(experiment))          #removes all the subdirectories!
    os.makedirs('Experiments/'+ str(experiment))
writer = pd.ExcelWriter('Experiments/'+ str(experiment) + '/df.xlsx') 



text_file = open('Experiments/'+ str(experiment) +'/parameters.txt', "w")
text_file.write('experiment: ' + str(experiment)+ '\n')
text_file.write('number of experiments: ' + str(number_of_experiment)+ '\n')
text_file.write('file: ' + str(file)+ '\n')
text_file.write('learner model: ' + str(learner_model)+ '\n')
text_file.write('episode size: ' + str(episode_size)+ '\n')
#text_file.write('numbers of epocs: ' + str(epocs)+ '\n')
text_file.write('internal trashold: ' + str(internal_trashold)+ '\n')
text_file.write('external trashold: ' + str(external_trashold)+ '\n')
 
text_file.close()

#### 4. Run all experiments

In [ ]:
for e in range (number_of_experiment):
    if not os.path.exists('Experiments/'+ str(experiment)+ '/'+ str(e)):
        os.makedirs('Experiments/'+ str(experiment)+ '/'+ str(e))
    else:
        shutil.rmtree('Experiments/'+ str(experiment)+ '/'+ str(e))          #removes all the subdirectories!
        os.makedirs('Experiments/'+ str(experiment)+ '/'+ str(e))
    print ('Experiments ' + str(e) + ' start')
##########################Experiment setup##########################
    # Read the data
    data = pd.read_csv(location + '/' + filename, index_col=0)
    #Set the number of iterations:
    interations=10*len(data.index)/episode_size
#    interations=2400
    # Set the number of columns exclude the class column
    number_of_columns=data.shape[1]-1 
    print ("number of columns: "+ str(number_of_columns) +" (exclude class column)" ) 
    # Set the number of episodes 
    # episodes_number=epocs*len(data.index)/episode_size
    episodes_number=interations
    print ("Number of episodes: "+ str(episodes_number) ) 
    # Initialize matrix Q as a 1 values matrix:
    #Q = np.matrix(np.ones([number_of_columns+1,number_of_columns+1])) # we will use the last dummy columns as initial state s
    Q = np.matrix(np.ones([number_of_columns+1,number_of_columns+1])) # we will use the last dummy columns as initial state s
    # Set initial_state to be the last dummy column we have created
    initial_state=number_of_columns
    # define data frame to save episode policies results
    df = pd.DataFrame(columns=('episode','episode_columns','policy_columns','policy_accuracy_train','policy_accuracy_test'))
    print ("initial state number: "+ str(initial_state) + " (the last dummy column we have created)") 
    ##########################  episode  ##########################  
    for i in range (int(episodes_number)):
    ########## Begining of episode  ############
        # Initiate lists for available_act, episode_columns and and the policy mode & episode_error
        episode_available_act=list(np.arange(number_of_columns))
        episode_columns=[]
        policy=0
        episode_error=0
        # Initiate the error to 0.5
        episode_last_error=0.5
        # Initiate current_state to be initial_state
        episode_current_state=initial_state
        # Create the episode data 
        episode= get_data(episode_size, policy=0, mode='train')
        # Separate the episode data into features and label
        X_episode,y_episode=data_separate(episode)
        # Split the data into train and test 
        X_train_main_episode, X_test_main_episode, y_train_episode, y_test_episode = data_split(X_episode,y_episode)
        if i<episodes_number*0.25:
            epsilon=0.9
            learning_rate=0.09
        elif i<episodes_number*0.5:
            epsilon=0.5
            learning_rate=0.05
        elif i<episodes_number*0.75:
            epsilon=0.3
            learning_rate=0.01
        else:
            epsilon=0.1
            learning_rate=0.005
        ########## Q learning start ############

        while len(episode_available_act)>0:
            # Get exploration or explotation flag 
            exploration=exploration_explotation(epsilon)
            # Get available actions in the current state
            episode_available_act = available_actions(number_of_columns,episode_columns,initial_state,episode_current_state,internal_trashold,exploration)
            if len(episode_available_act)>0:
                # Sample next action to be performed
                episode_action = sample_next_action(episode_current_state, Q, episode_available_act, exploration)
                # Update the episode_columns
                episode_columns=update_columns(episode_action,episode_columns)
                # Update the dataset to include all episode columns + current selected action (column)
                X_train_episode, X_test_episode =update_X_train_X_test(episode_columns,X_train_main_episode, X_test_main_episode)
                # Update the accuracy of the current columns
                episode_error= Learner(X_train_episode, X_test_episode, y_train_episode, y_test_episode)
                # Update reward
                episode_reward=episode_last_error-episode_error
                # Update Q matrix
                q_update(episode_current_state,episode_action,learning_rate, episode_reward)
                # Update parameters for next round 
#                 if episode_current_state==initial_state:
#                     beta=abs(episode_reward-Q[episode_current_state,episode_action])
#                     epsilon=final_epsilon+(beta*(1-final_epsilon))
                #    learning_rate=final_learning_rate+(beta*(1-final_learning_rate))
                episode_current_state=episode_action
                episode_last_error=episode_error
                 
        ########## Q learning End ############

        #Save Q matrix: 
        if (i%100 ==0):
            Q_save=pd.DataFrame(Q)
            Q_save.to_csv('Experiments/'+ str(experiment)+ '/'+ str(e)+ '/Q.'+ str(i+1) + '.csv') 

        # Calculate policy 
        policy_available_actions=list(np.arange(number_of_columns))
        policy_columns=[]
        policy_current_state=initial_state
        while len(policy_available_actions)>0:
            # Get available actions in the current state
            policy_available_actions = available_actions(number_of_columns,policy_columns,initial_state,policy_current_state, external_trashold, exploration=0)
            # # Sample next action to be performed
            if len(policy_available_actions)>0:
                policy_select_action = sample_next_action(policy_current_state, Q, policy_available_actions, exploration=0)
                # Update the episode_columns
                policy_columns=update_columns(policy_select_action,policy_columns)
                policy_current_state=policy_select_action
        # Calculate policy_accuracy    
        if len(policy_columns)>0:
            ##for training dataset##
            policy_data=get_data(episode_size,policy=1,mode='train')
            X_policy,y_policy=data_separate(policy_data)
            X_train_main_policy, X_test_main_policy, y_train_policy, y_test_policy = data_split(X,y)
            X_train_policy, X_test_policy =update_X_train_X_test(policy_columns, X_train_main_policy, X_test_main_policy)
            policy_error=Learner(X_train_policy, X_test_policy,y_train_policy, y_test_policy)
            policy_accuracy_train=1-policy_error
            ##for testing dataset##
            policy_data=get_data(episode_size,policy=1,mode='test')
            X_policy,y_policy=data_separate(policy_data)
            X_train_main_policy, X_test_main_policy, y_train_policy, y_test_policy = data_split(X,y)
            X_train_policy, X_test_policy =update_X_train_X_test(policy_columns, X_train_main_policy, X_test_main_policy)
            policy_error=Learner(X_train_policy, X_test_policy,y_train_policy, y_test_policy)
            policy_accuracy_test=1-policy_error 
        else:
            policy_accuracy_train=0 
            policy_accuracy_test=0
        df=df.append({'episode':str(i+1), 'episode_columns':str(episode_columns),'policy_columns':str(policy_columns),'policy_accuracy_train':policy_accuracy_train,'policy_accuracy_test':policy_accuracy_test}, ignore_index=True)
        #Prints
        print ("episode "+ str(i+1) +" start") 
        print ("episode columns: "+ str(episode_columns) + " epsilon: " + str(epsilon) + " learning rate: " + str(learning_rate) + " error: " +str(episode_error))
        print ("episode policy:" + str(policy_columns) + " train accuracy: " + str(policy_accuracy_train)  + " test accuracy: " +str(policy_accuracy_test)) 
        print ("episode "+ str(i+1) +" end") 
    ########## End of episode  ############
    df.to_excel(writer, 'Experiment' + str(e))
    df_plot=df[['episode','policy_accuracy_train','policy_accuracy_test']]
    plot=df_plot.plot()
    fig = plot.get_figure()
    fig.savefig('Experiments/'+ str(experiment) + '/plot_experiment_' + str(e) +'.png')
writer.save()

Experiments 0 start
number of columns: 12 (exclude class column)
Number of episodes: 27000.0
initial state number: 12 (the last dummy column we have created)
episode 1 start
episode columns: [2, 5, 3, 4, 11, 6, 9, 8, 0, 7, 10, 1] epsilon: 0.9 learning rate: 0.09 error: 0.3
episode policy:[11, 10, 0, 4, 3, 5, 7, 6, 1, 8, 2, 9] train accuracy: 0.62012962963 test accuracy: 0.584166666667
episode 1 end
episode 2 start
episode columns: [4, 8, 6, 7, 2, 0, 9, 3, 5, 10, 11, 1] epsilon: 0.9 learning rate: 0.09 error: 0.45
episode policy:[0, 3, 1, 6, 8, 10, 5, 2, 9, 7, 4] train accuracy: 0.677314814815 test accuracy: 0.6165
episode 2 end
episode 3 start
episode columns: [9, 6, 0, 5, 4, 7, 3, 8, 10, 1, 2, 11] epsilon: 0.9 learning rate: 0.09 error: 0.35
episode policy:[5, 6, 2, 4, 1, 11, 0, 3, 10, 9, 7, 8] train accuracy: 0.619740740741 test accuracy: 0.587333333333
episode 3 end
episode 4 start
episode columns: [7, 4, 8, 10, 6, 2, 5, 3, 0, 1, 11, 9] epsilon: 0.9 learning rate: 0.09 error: 0.3
ep

episode 35 start
episode columns: [6, 4, 0, 10, 5, 2, 8, 9, 3, 11, 1, 7] epsilon: 0.9 learning rate: 0.09 error: 0.45
episode policy:[6, 10, 3, 7, 1, 0, 2, 5, 8, 9, 11, 4] train accuracy: 0.619666666667 test accuracy: 0.584333333333
episode 35 end
episode 36 start
episode columns: [4, 11, 10, 7, 3, 1, 6, 0, 8, 5, 2, 9] epsilon: 0.9 learning rate: 0.09 error: 0.5
episode policy:[6, 10, 3, 7, 1, 0, 2, 5, 8, 9, 11, 4] train accuracy: 0.619333333333 test accuracy: 0.583
episode 36 end
episode 37 start
episode columns: [6, 9, 7, 3, 0, 4, 10, 1, 5, 2, 8, 11] epsilon: 0.9 learning rate: 0.09 error: 0.5
episode policy:[6, 10, 1, 0, 2, 5, 8, 9, 11, 4, 7] train accuracy: 0.621814814815 test accuracy: 0.5835
episode 37 end
episode 38 start
episode columns: [6, 10, 1, 11, 8, 2, 9, 4, 5, 7, 3, 0] epsilon: 0.9 learning rate: 0.09 error: 0.35
episode policy:[6, 10, 3, 7, 1, 0, 2, 5, 8, 9, 11, 4] train accuracy: 0.620296296296 test accuracy: 0.5855
episode 38 end
episode 39 start
episode columns: [11,

episode 70 start
episode columns: [3, 11, 0, 1, 5, 2, 8, 9, 7, 10, 6, 4] epsilon: 0.9 learning rate: 0.09 error: 0.4
episode policy:[2, 10, 7, 8, 0, 1, 5, 4, 9, 11, 3, 6] train accuracy: 0.619722222222 test accuracy: 0.585166666667
episode 70 end
episode 71 start
episode columns: [8, 2, 10, 6, 3, 5, 4, 9, 1, 0, 7, 11] epsilon: 0.9 learning rate: 0.09 error: 0.5
episode policy:[2, 10, 7, 8, 0, 1, 5, 4, 9, 11, 3, 6] train accuracy: 0.6195 test accuracy: 0.585666666667
episode 71 end
episode 72 start
episode columns: [9, 2, 10, 1, 7, 6, 5, 11, 3, 4, 0, 8] epsilon: 0.9 learning rate: 0.09 error: 0.25
episode policy:[2, 10, 7, 8, 0, 1, 5, 4, 9, 11, 3, 6] train accuracy: 0.619740740741 test accuracy: 0.585833333333
episode 72 end
episode 73 start
episode columns: [11, 10, 1, 2, 5, 4, 6, 8, 3, 9, 7, 0] epsilon: 0.9 learning rate: 0.09 error: 0.3
episode policy:[2, 10, 7, 8, 0, 1, 5, 4, 9, 11, 3, 6] train accuracy: 0.620203703704 test accuracy: 0.586166666667
episode 73 end
episode 74 start
ep

episode 105 start
episode columns: [1, 8, 3, 6, 4, 9, 7, 2, 11, 5, 0, 10] epsilon: 0.9 learning rate: 0.09 error: 0.2
episode policy:[2, 9, 0, 10, 7, 6, 4, 8, 1, 5] train accuracy: 0.681259259259 test accuracy: 0.621166666667
episode 105 end
episode 106 start
episode columns: [0, 8, 11, 9, 6, 5, 1, 4, 7, 3, 10, 2] epsilon: 0.9 learning rate: 0.09 error: 0.55
episode policy:[2, 9, 0, 10, 7, 6, 4, 8, 1, 5] train accuracy: 0.681333333333 test accuracy: 0.621333333333
episode 106 end
episode 107 start
episode columns: [6, 4, 10, 9, 0, 3, 5, 8, 7, 1, 2, 11] epsilon: 0.9 learning rate: 0.09 error: 0.2
episode policy:[2, 11, 10, 7, 6, 4, 0, 3, 1, 5] train accuracy: 0.623666666667 test accuracy: 0.5875
episode 107 end
episode 108 start
episode columns: [5, 7, 9, 3, 1, 4, 8, 11, 10, 6, 2] epsilon: 0.9 learning rate: 0.09 error: 0.3
episode policy:[2, 11, 10, 7, 6, 4, 0, 3, 9, 8, 1, 5] train accuracy: 0.619740740741 test accuracy: 0.585
episode 108 end
episode 109 start
episode columns: [9, 6, 5

episode 139 start
episode columns: [3, 2, 4, 5, 11, 10, 7, 0, 9, 8, 6, 1] epsilon: 0.9 learning rate: 0.09 error: 0.2
episode policy:[6, 4, 0, 2, 3, 7, 5, 1, 8, 9, 10] train accuracy: 0.677222222222 test accuracy: 0.617
episode 139 end
episode 140 start
episode columns: [5, 9, 8, 7, 1, 3, 11, 10, 2, 6, 0, 4] epsilon: 0.9 learning rate: 0.09 error: 0.25
episode policy:[6, 4, 0, 2, 3, 7, 5, 1, 8, 9, 10] train accuracy: 0.677314814815 test accuracy: 0.616666666667
episode 140 end
episode 141 start
episode columns: [6, 4, 10, 0, 11, 5, 7, 2, 8, 1, 9, 3] epsilon: 0.9 learning rate: 0.09 error: 0.45
episode policy:[2, 6, 4, 0, 3, 7, 5, 1, 8, 9, 10] train accuracy: 0.677388888889 test accuracy: 0.616
episode 141 end
episode 142 start
episode columns: [2, 4, 8, 1, 10, 11, 3, 9, 7, 6, 0, 5] epsilon: 0.9 learning rate: 0.09 error: 0.35
episode policy:[2, 6, 4, 0, 3, 7, 5, 1, 8, 9, 10] train accuracy: 0.677166666667 test accuracy: 0.617333333333
episode 142 end
episode 143 start
episode columns: 

...

episode 7546 start
episode columns: [3, 9, 4, 2, 6, 0, 8, 7, 1, 5, 10] epsilon: 0.5 learning rate: 0.05 error: 0.6
episode policy:[4, 2, 6, 9, 7, 1, 5, 3, 8, 10] train accuracy: 0.695574074074 test accuracy: 0.682166666667
episode 7546 end
episode 7547 start
episode columns: [8, 11, 5, 0, 4, 2, 1, 9, 7, 6, 3, 10] epsilon: 0.5 learning rate: 0.05 error: 0.45
episode policy:[4, 2, 6, 3, 9, 7, 1, 5] train accuracy: 0.696037037037 test accuracy: 0.685666666667
episode 7547 end
episode 7548 start
episode columns: [4, 7, 6, 0, 8, 9] epsilon: 0.5 learning rate: 0.05 error: 0.5
episode policy:[4, 2, 6, 3, 9, 7, 1, 5] train accuracy: 0.696037037037 test accuracy: 0.685666666667
episode 7548 end
episode 7549 start
episode columns: [4, 2, 6, 9, 11, 7, 3, 8, 0] epsilon: 0.5 learning rate: 0.05 error: 0.4
episode policy:[4, 9, 7, 6, 3, 8, 2] train accuracy: 0.695962962963 test accuracy: 0.688166666667
episode 7549 end
episode 7550 start
episode columns: [4, 8, 10, 6, 0, 7, 11, 5, 9, 2, 3, 1] epsilo

episode 7583 start
episode columns: [4, 9, 6, 11, 5, 10, 1, 7, 8, 0, 2, 3] epsilon: 0.5 learning rate: 0.05 error: 0.25
episode policy:[4, 9, 6, 3, 7, 1, 5] train accuracy: 0.696148148148 test accuracy: 0.6875
episode 7583 end
episode 7584 start
episode columns: [4, 9, 6, 0, 8, 10, 1, 5, 2, 3, 11, 7] epsilon: 0.5 learning rate: 0.05 error: 0.55
episode policy:[4, 9, 6, 3, 7, 1, 5] train accuracy: 0.696148148148 test accuracy: 0.6875
episode 7584 end
episode 7585 start
episode columns: [4, 11, 5, 3, 7, 0, 8, 9, 6, 10, 2] epsilon: 0.5 learning rate: 0.05 error: 0.6
episode policy:[4, 9, 6, 3, 7, 1, 5] train accuracy: 0.696148148148 test accuracy: 0.6875
episode 7585 end
episode 7586 start
episode columns: [4, 3, 10, 1, 5, 6, 0, 9, 11, 2] epsilon: 0.5 learning rate: 0.05 error: 0.35
episode policy:[4, 9, 6, 3, 7, 1, 5] train accuracy: 0.696148148148 test accuracy: 0.6875
episode 7586 end
episode 7587 start
episode columns: [4, 9, 6, 3, 11, 2] epsilon: 0.5 learning rate: 0.05 error: 0.45
e

episode 7621 start
episode columns: [4, 11, 2, 3, 10, 6, 8, 9, 1, 5, 0, 7] epsilon: 0.5 learning rate: 0.05 error: 0.4
episode policy:[8, 9, 7, 1, 10, 4, 3, 6, 5] train accuracy: 0.696185185185 test accuracy: 0.685833333333
episode 7621 end
episode 7622 start
episode columns: [8, 9, 7, 2, 3, 10, 1, 11, 0, 4, 6, 5] epsilon: 0.5 learning rate: 0.05 error: 0.6
episode policy:[8, 9, 7, 1, 10, 4, 6, 3] train accuracy: 0.696351851852 test accuracy: 0.688833333333
episode 7622 end
episode 7623 start
episode columns: [8, 9, 5, 11, 2, 3, 10, 4, 7, 0] epsilon: 0.5 learning rate: 0.05 error: 0.45
episode policy:[8, 9, 7, 1, 10, 4, 6, 3] train accuracy: 0.696333333333 test accuracy: 0.688833333333
episode 7623 end
episode 7624 start
episode columns: [1, 10, 9, 7, 6, 8, 4, 3, 5, 11, 2] epsilon: 0.5 learning rate: 0.05 error: 0.35
episode policy:[8, 9, 7, 1, 10, 4, 6, 3] train accuracy: 0.696351851852 test accuracy: 0.688833333333
episode 7624 end
episode 7625 start
episode columns: [6, 2, 0, 9, 10,

episode 7657 start
episode columns: [8, 4, 11, 6, 1, 10, 9, 5, 0, 7, 3] epsilon: 0.5 learning rate: 0.05 error: 0.35
episode policy:[8, 9, 7, 6, 4, 3, 10] train accuracy: 0.696518518519 test accuracy: 0.689166666667
episode 7657 end
episode 7658 start
episode columns: [3, 2, 11, 0, 9, 7, 6, 1, 5, 4] epsilon: 0.5 learning rate: 0.05 error: 0.45
episode policy:[8, 9, 7, 6, 4, 3, 10] train accuracy: 0.696518518519 test accuracy: 0.689166666667
episode 7658 end
episode 7659 start
episode columns: [7, 6, 4, 3, 10, 9, 1, 2, 5, 0, 8, 11] epsilon: 0.5 learning rate: 0.05 error: 0.45
episode policy:[8, 9, 7, 6, 4, 3, 10] train accuracy: 0.696518518519 test accuracy: 0.689166666667
episode 7659 end
episode 7660 start
episode columns: [11, 6, 4, 1, 8, 5] epsilon: 0.5 learning rate: 0.05 error: 0.25
episode policy:[8, 9, 7, 6, 4, 3, 10] train accuracy: 0.696518518519 test accuracy: 0.689166666667
episode 7660 end
episode 7661 start
episode columns: [6, 4, 5, 8, 0, 7, 2, 9] epsilon: 0.5 learning ra

episode 7693 start
episode columns: [8, 7, 3, 0, 11, 9, 1, 2, 6, 10, 4] epsilon: 0.5 learning rate: 0.05 error: 0.4
episode policy:[8, 9, 7, 3, 10, 4, 6, 1, 5] train accuracy: 0.696185185185 test accuracy: 0.685833333333
episode 7693 end
episode 7694 start
episode columns: [8, 9, 7, 6, 10, 4, 3, 2, 5, 11, 1, 0] epsilon: 0.5 learning rate: 0.05 error: 0.4
episode policy:[8, 9, 7, 3, 10, 4, 6, 1, 5] train accuracy: 0.696185185185 test accuracy: 0.685833333333
episode 7694 end
episode 7695 start
episode columns: [8, 9, 11, 6, 5, 7, 3, 10, 4, 2] epsilon: 0.5 learning rate: 0.05 error: 0.35
episode policy:[8, 9, 7, 3, 10, 4, 6, 1, 5] train accuracy: 0.696185185185 test accuracy: 0.685833333333
episode 7695 end
episode 7696 start
episode columns: [4, 1, 9, 8, 10, 6, 2, 5] epsilon: 0.5 learning rate: 0.05 error: 0.4
episode policy:[8, 9, 7, 3, 10, 4, 6, 1, 5] train accuracy: 0.696185185185 test accuracy: 0.685833333333
episode 7696 end
episode 7697 start
episode columns: [9, 7, 8, 10, 4, 6, 5